In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Text, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

In [2]:
engine = create_engine('sqlite:///crm.db', echo=False)

In [3]:
Base = declarative_base()

In [4]:
class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    last_name = Column(String)
    email = Column(String)

    def __repr__(self):
        return "<Customer(name='{}', last_name='{}', email='{}')>".format(
        self.name, self.last_name, self.email)

In [5]:
Base.metadata.create_all(engine)

In [6]:
first_customer = Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')

In [7]:
first_customer

<Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')>

In [8]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
session.add(first_customer)
session.commit()

In [10]:
one_customer = session.query(Customer).filter_by(name='Aquiles').first() 

In [11]:
print(one_customer)

<Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')>


In [12]:
one_customer = session.query(Customer).filter(Customer.name.like('aqui%')).first() 

In [13]:
print(one_customer)

<Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')>


In [14]:
second_customer = Customer(name='Aquileo', last_name='Doe', email='aquileo@doe.com')
session.add(second_customer)
session.commit()

In [15]:
answer = session.query(Customer).filter(Customer.name.like('aqui%'))

In [16]:
answer

In [17]:
for c in answer:
    print(c)

<Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')>
<Customer(name='Aquileo', last_name='Doe', email='aquileo@doe.com')>


In [18]:
class Message(Base):
    __tablename__ = 'messages'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    text = Column(Text)
    date = Column(Date)
    
    customer_id = Column(Integer, ForeignKey('customers.id'))
    customer = relationship("Customer", back_populates="messages")

    def __repr__(self):
        return "<Message(name='{}', date='{}', customer='{}')>".format(
        self.name, self.date, self.customer)

In [19]:
Customer.messages = relationship('Message', order_by=Message.id, back_populates='customer')

In [20]:
Base.metadata.create_all(engine)

In [21]:
from datetime import datetime

In [22]:
c = session.query(Customer).first()

In [23]:
new_message = Message(name='Welcome', text='Welcome to the new CRM', date=datetime.now(), customer=c)

In [24]:
session.add(new_message)
session.commit()

In [25]:
message = session.query(Message).first()
print(message)

<Message(name='Welcome', date='2019-02-04', customer='<Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')>')>


In [26]:
c.messages

[<Message(name='Welcome', date='2019-02-04', customer='<Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')>')>]

## Introducing Many-to-many

In [27]:
class List(Base):
    __tablename__ = 'lists'
    id = Column(Integer, primary_key=True)
    name = Column(String)
Base.metadata.create_all(engine)

In [28]:
from sqlalchemy import Table

association_table = Table('list_customer', Base.metadata,
    Column('left_id', Integer, ForeignKey('customers.id')),
    Column('right_id', Integer, ForeignKey('lists.id'))
)

In [29]:
Base.metadata.create_all(engine)

In [30]:
Customer.lists = relationship("List",
                    secondary=association_table,
                    backref="customers")

In [31]:
# List.customers = relationship("Customer", secondary=association_table, backref='lists')

In [32]:
Base.metadata.create_all(engine)

In [33]:
c_list = List(name='New List')

In [34]:
c_list.customers.append(c)

In [35]:
session.add(c_list)
session.commit()

In [36]:
las_list = session.query(List).first()

In [37]:
las_list.customers

[<Customer(name='Aquiles', last_name='Carattino', email='aquiles@uetke.com')>]

In [38]:
c.lists